In [ ]:
# Login to Azure
Write-Host "Checking if logged in to Azure..."

$loggedIn = az account show --query "name" -o tsv

if ($loggedIn -ne $null) {
    Write-Host "Already logged in as $loggedIn"
} else {
    Write-Host "Logging in..."
    az login
}

# Retrieve the default subscription ID
$subscriptionId = (
    (
        az account list -o json `
            --query "[?isDefault]"
    ) | ConvertFrom-Json
).id

# Set the subscription
az account set --subscription $subscriptionId
Write-Host "Subscription set to $subscriptionId"

In [ ]:
# Run Deploy-Infrastructure.ps1
.\Deploy-Infrastructure.ps1 -DeploymentName 'docintel-gpt-document-extraction' -Location 'eastus'

In [ ]:
#r "nuget:System.Text.Json, 8.0.1"
#r "nuget:DotNetEnv, 3.0.0"
#r "nuget:Azure.AI.OpenAI, 1.0.0-beta.14"
#r "nuget:Azure.AI.DocumentIntelligence, 1.0.0-beta.2"

In [5]:
using System.Net;
using System.Net.Http;
using System.Text.Json.Nodes;
using System.Text.Json;
using System.IO; 

using Azure;
using Azure.AI.DocumentIntelligence;
using Azure.AI.OpenAI;
using DotNetEnv;

In [7]:
Env.Load("config.env");

var openAIEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var openAIApiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
var openAIModelDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_MODEL_DEPLOYMENT_NAME");
var openAIApiVersion = "2023-12-01-preview";
var documentIntelligenceEndpoint = Environment.GetEnvironmentVariable("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT");
var documentIntelligenceApiKey = Environment.GetEnvironmentVariable("AZURE_DOCUMENT_INTELLIGENCE_KEY");

var pdfName = "Invoice_1.pdf";

var documentIntelligenceClient = new DocumentIntelligenceClient(new Uri(documentIntelligenceEndpoint), new AzureKeyCredential(documentIntelligenceApiKey));
var openAIClient = new OpenAIClient(new Uri(openAIEndpoint), new AzureKeyCredential(openAIApiKey));

In [8]:
var markdownAnalysisContent = new AnalyzeDocumentContent()
{
    Base64Source = BinaryData.FromBytes(File.ReadAllBytes(pdfName))
};

Operation<AnalyzeResult> markdownAnalysisOperation = await documentIntelligenceClient.AnalyzeDocumentAsync(WaitUntil.Completed, "prebuilt-layout", markdownAnalysisContent, outputContentFormat: ContentFormat.Markdown);
var markdown = markdownAnalysisOperation.Value.Content;

In [9]:
var jsonStructure = new {
    company_name = "",
    invoice_date = "",
    products = new [] {
        new {
            id = "",
            unit_price = "",
            quantity = "",
            total = ""
        }
    },
    total_amount = "",
    signatures = new [] {
        new {
            type = "",
            has_signature = "",
            signed_on = ""
        }
    }
};

ChatCompletionsOptions options = new ChatCompletionsOptions()
{
    DeploymentName = openAIModelDeployment,
    MaxTokens = 4096,
    Temperature = 0.1f,
    NucleusSamplingFactor = 0.1f
};

options.Messages.Add(new ChatRequestSystemMessage("You are an AI assistant that extracts data from documents and returns them as structured JSON objects. Do not return as a code block."));
options.Messages.Add(new ChatRequestUserMessage($"Extract the data from this invoice. If a value is not present, provide null. Use the following structure: {JsonSerializer.Serialize(jsonStructure)}"));
options.Messages.Add(new ChatRequestUserMessage(markdown));

Response<ChatCompletions> response = await openAIClient.GetChatCompletionsAsync(options);
foreach(var completion in response.Value.Choices)
{
    Console.WriteLine(completion.Message.Content);
}

{
  "company_name": "CONTOSO",
  "invoice_date": "2/27/2024",
  "products": [
    {
      "id": "5-01-XX",
      "unit_price": "1.00",
      "quantity": "1.0",
      "total": "1.00"
    },
    {
      "id": "5-02-XX",
      "unit_price": "1.50",
      "quantity": "5.0",
      "total": "7.50"
    },
    {
      "id": "5-03-XX",
      "unit_price": "5.75",
      "quantity": "2.0",
      "total": "11.50"
    },
    {
      "id": "5-04-XX",
      "unit_price": "2.80",
      "quantity": "6.0",
      "total": "16.80"
    },
    {
      "id": "5-05-XX",
      "unit_price": "4.45",
      "quantity": "13.0",
      "total": "57.85"
    },
    {
      "id": "5-06-XX",
      "unit_price": "2.20",
      "quantity": "11.0",
      "total": "24.20"
    },
    {
      "id": "5-07-XX",
      "unit_price": "20.05",
      "quantity": "5.0",
      "total": "100.25"
    },
    {
      "id": "5-08-XX",
      "unit_price": "9.50",
      "quantity": "1.0",
      "total": "9.50"
    },
    {
      "id": "5-09-X